# Profitability Analysis

This notebook evaluates the company's profitability, margins, returns, and
valuation metrics to assess business quality and earnings power.


In [1]:
from src.setup import *

## Step 1: Metric Categories

We compute the following groups of metrics:

- **Valuation**: P/E, Earnings Yield, PEG
- **Margins**: Gross, Net, EBIT, EBITDA
- **Returns**: ROE, ROA, ROIC
- **Cash Flow**: Free Cash Flow Yield, Operating Leverage
- **Growth**: EPS Growth, Free Cash Flow Growth

In [2]:

def profitability_analysis(df, annual_avg, info, balance_sheet, ticker, market_cap, price, shares_outstanding, tax_rate, eps_growth_forward=None):
    """
    Compute profitability, margin, return and valuation metrics.

    Parameters:
    ----------
    df : pandas.DataFrame
        Quarterly balance sheet data.
    annual_avg : pandas.DataFrame
        Annual averages of key metrics, e.g., FCF, EPS.
    info : yfinance.Ticker
        Financial information from Yahoo Finance.
    balance_sheet : pandas.DataFrame
        Latest balance sheet data.
    ticker : str
        Stock ticker symbol.
    market_cap : float
        Current market capitalization.
    price : float
        Current stock price.
    shares_outstanding : float
        Latest number of shares outstanding.
    tax_rate : float
        Corporate tax rate in percent.
    eps_growth_forward : float, optional
        Forward EPS growth for PEG calculation (in percent).

    Returns
    -------
    pd.DataFrame
        Key profitability and valuation metrics.
    """

    # --- Core financials ---
    revenue = info.financials.loc["Total Revenue"].iloc[0] # Latest revenue
    gross_profit = info.financials.loc["Gross Profit"].iloc[0] # Latest gross profit
    net_income = info.financials.loc["Net Income"].iloc[0] # Latest net income  
    ebit = info.financials.loc["EBIT"].iloc[0] # Latest EBIT
    ebitda = info.financials.loc["EBITDA"].iloc[0] # Latest EBITDA
    free_cashflow = annual_avg[f"{ticker}: Annual Free Cash Flow"].iloc[-1] # Latest FCF

    equity = df[f"{ticker}: Quarterly Shareholders Equity"].iloc[-1] # Latest equity value
    debt = balance_sheet.loc["Total Debt"].iloc[0] # Latest debt
    cash = annual_avg[f"{ticker}: Quarterly Cash & Cash Equivalents"].iloc[-1] # Latest cash

    # --- Valuation ---
    eps = net_income / shares_outstanding if shares_outstanding != 0 else np.nan #calculate EPS
    pe_ratio = price / eps if eps != 0 else np.nan 
    earnings_yield = eps / price if price != 0 else np.nan

    # --- Margins ---
    gross_margin = gross_profit / revenue if revenue != 0 else np.nan
    net_margin = net_income / revenue if revenue != 0 else np.nan
    ebit_margin = ebit / revenue if revenue != 0 else np.nan
    ebitda_margin = ebitda / revenue if revenue != 0 else np.nan

    # --- Returns ---
    total_assets = equity + debt
    nopat = ebit * (1 - tax_rate / 100)
    invested_capital = total_assets - cash

    roe = net_income / equity if equity != 0 else np.nan # Return on Equity
    roa = net_income / total_assets if total_assets != 0 else np.nan # Return on Assets
    roic = nopat / invested_capital if invested_capital != 0 else np.nan # Return on Invested Capital

    # --- Cashflow metrics ---
    fcf_yield = free_cashflow / market_cap if market_cap != 0 else np.nan
    operating_leverage = ebit / gross_profit if gross_profit != 0 else np.nan

    # --- EPS growth (trailing) ---
    eps_series = annual_avg[f"{ticker}: Annual EPS"].dropna()

    trailing_growth = None
    trailing_peg = None
    if len(eps_series) >= 4:
        growth_rates = [
            eps_series.iloc[-1] / eps_series.iloc[-2] - 1,
            eps_series.iloc[-2] / eps_series.iloc[-3] - 1,
            eps_series.iloc[-3] / eps_series.iloc[-4] - 1,
        ]
        trailing_growth = np.mean(growth_rates)
        trailing_peg = pe_ratio / (trailing_growth * 100) # multiply by 100 to convert to percent

    # --- Forward PEG ---
    forward_peg = None
    if eps_growth_forward is not None:
        forward_peg = pe_ratio / eps_growth_forward

    # --- Free Cashflow Growth ---
    fcf_growth = None
    fcf_series = annual_avg.get(f"{ticker}: Annual Free Cash Flow", pd.Series([np.nan]))
    if len(fcf_series) >= 2:
        prev_fcf = fcf_series.iloc[-2]
        fcf_growth = (free_cashflow / prev_fcf - 1) if prev_fcf != 0 else np.nan


    # --- Output ---
    metrics = {
        "EPS": eps,
        "P/E Ratio": pe_ratio,
        "Earnings Yield": earnings_yield,
        "Gross Margin": gross_margin,
        "Net Margin": net_margin,
        "EBIT Margin": ebit_margin,
        "EBITDA Margin": ebitda_margin,
        "Free Cash Flow Yield": fcf_yield,
        "Operating Leverage": operating_leverage,
        "Return on Equity (ROE)": roe,
        "Return on Assets (ROA)": roa,
        "ROIC": roic,
        "Trailing PEG Ratio": trailing_peg,
        "Forward PEG Ratio": forward_peg,
        "Average EPS Growth (3 Years)": trailing_growth,
        "Free Cashflow Growth (1 Year)": fcf_growth,
        "Total Assets": total_assets,
        "Revenue": revenue,
        "Net Income": net_income,
        "EBIT": ebit,
        "EBITDA": ebitda
    }
     
    df_metrics = pd.DataFrame.from_dict(metrics, orient="index", columns=["Value"])
    return df_metrics



## Step 2: Run Profitability Analysis

We now compute all profitability and valuation metrics using the latest financial data.

In [3]:
# --- Example Usage ---

df_profit = profitability_analysis(
    df=df,
    annual_avg=annual_avg,
    info=info,
    balance_sheet=balance_sheet,
    ticker=TICKER,
    market_cap=market_cap,
    price=price,
    shares_outstanding=shares_outstanding,
    tax_rate=TAX_RATE,
    eps_growth_forward=8.0)  # optional, in %

## Step 3: Results

The table below summarizes the company's profitability, returns, margins, and valuation metrics.


In [4]:
display(df_profit.style.format({"Value": "{:,.2f}"}))

,Value
EPS,13.70
P/E Ratio,32.41
Earnings Yield,0.03
Gross Margin,0.69
Net Margin,0.36
EBIT Margin,0.45
EBITDA Margin,0.57
Free Cash Flow Yield,0.03
Operating Leverage,0.65
Return on Equity (ROE),0.28
